# Definições

Este espaço é dedicado para a importação de bibliotecas utilizadas em todas as etapas: listagem de títulos, download da fonte e processamento de informações (*scrap*). Além disso, duas variáveis importantes são inicializadas: **BASE_PATH** e **publishers**.

A variável **BASE_PATH** tem como principal função gerenciar os caminhos para salvar arquivos de dados e fontes. Com ela, independente do sistema operacional, é possível fazer com que os arquivos sejam posicionados nos diretórios corretos.

Por outro lado **publishers** é a variável que controla as editoras de quadrinhos que terão seu portfólio listado, já que a primeira parte (listagem de títulos) ocorre por meio da pesquisa avançada, utilizando um *query param* para filtrar resultados para uma editora. Isto resulta em listagens menores do que uma listagem ampla, reduzindo casos de truncagem por meio da rota de busca.

In [29]:
import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, datetime
from selenium.webdriver.common.by import By


BASE_PATH = os.path.dirname(os.getcwd())

In [3]:
publishers = [
    "pipoca+e+nanquim",
    "newpop",
    "jbc",
    "panini"
]

# Parte 1: Listagem de Títulos

Como muitas lojas possuem restrições de *requests* feitas automaticamente, um RPA é desenvolvido para acessar a rota de pesquisa e listar todos os títulos encontrados, junto com os seus links para páginas de detalhes. Este processo inicial tem como objetivo obter uma diretiva para saber quais páginas de quais títulos serão acessadas para captura de dados.

O resultado desta etapa é um conjunto de dados com os títulos disponíveis e o link para acessar a página de detalhes. Este conjunto é criado em um tempo relativamente curto e precisa de alguns *sleep* para permitir que o JS execute e a página seja completamente criada (a loja não provê páginas estáticas). O tempo de execução é a parte fundamental para esta etapa, pois a próxima é extremamante lenta e possuir uma listagem prévia de quais títulos serão acessados é de grande importância.

In [10]:
title_dataset = []
driver = webdriver.Chrome()

for publisher in publishers:
    url = f"https://www.amazon.com.br/s?i=stripbooks&rh=n%3A7842710011%2Cp_30%3A{publisher}&s=date-desc-rank&Adv-Srch-Books-Submit.x=38"
    driver.get(url)
    time.sleep(2)

    while True:
        # Recupera lista de resultados
        elements = driver.find_element(By.CLASS_NAME, "s-result-list")
        books = elements.find_elements(By.XPATH, ".//div[@data-component-type='s-search-result']")

        # Percorre os livros encontrados e salva dados
        for book in books:
            row = {"publisher": " ".join(publisher.split("+")), "extract_at": date.today().isoformat()}

            book_link = book.find_element(By.XPATH, ".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
            row["page_link"] = book_link.get_attribute("href")

            book_name = book.find_element(By.XPATH, ".//span[@class='a-size-medium a-color-base a-text-normal']")
            row["name"] = book_name.get_attribute("innerHTML")

            title_dataset.append(row)

        # Faz a troca de página, utilizando o botão de "próxima"
        old_url = driver.current_url
        driver.find_element(By.CLASS_NAME, "s-pagination-next").click()
        time.sleep(3)

        if driver.current_url == old_url:
            break

title_dataset = pd.DataFrame(title_dataset)
driver.close()

# Salva os dados primários para utilização posterior
title_dataset.to_csv(f"{BASE_PATH}/data/hqs/titles.csv", index=False)

# Parte 2: Download de Fonte

Como estamos lidando com *webscraping*, muitas lojas criam barreiras para acesso de scripts com requisições de acesso em alto volume. Alguns casos são contornados por meio da edição do *header* dentro da *request* junto de um delay entre cada requisição. Porém, ainda assim existem formas de bloquear tais acessos, o que acontece neste caso. Desta forma, outro RPA foi criado, que automatiza o processo de abertura dos links em um navegador e faz o download do HTML fonte da página.

O processo de abrir o link, recuperar o código fonte e salvar em um arquivo .html localmente é lento, principalmente se comparado com o método tradicional utilizando o *requests*. Todavia, como não estamos lidando com volumes altos de dados (são cerca de 3000 quadrinhos encontrados na etapa anterior) e como não existe necessidade de redownload, este processo ainda é considerado tratável, ou pelo menos "suportável".

Foi brevemente comentado que a listagem prévia seria importante nesta parte e o motivo é para a manutenção do *log*. Ao se trabalhar com páginas da internet e processos lento, manter um *log* de execução para ser possível identificar o que foi feito e o que falta fazer é de necessidade absoluta. Em qualquer falha no processo, seja por não carregamento, falha de memória ou até mesmo a interrupção manual, as páginas baixadas até o momento não serão revisitadas em uma próxima execução. Isto também permite que a Parte 1 seja atualizada sem que a Parte 2 precise reexecutar todo o download. Em termos mais específicos, o download é feito no modo *append*, sempre baixando coisas novas e nunca avaliando estruturas já baixadas. Novamente, este modo de operação é viável porque os dados que queremos são físicos (preço de capa, editora, formato e número de páginas) que não se alteram com o tempo.

O *log* de download é armazenado em um arquivo .txt que contêm apenas o título do produto processado e o tempo gasto no processo de abrir o navegador, acessar o link, baixar o código fonte e salvar em arquivo .html. Mesmo que o processo de atualização do arquivo de *log* seja uma atividade extra que pode tornar o processo mais lento, ele ainda é melhor do que assumir o risco de falha e ser necessário reiniciar a etapa lenta do início.

Por fim, a necessidade de fazer o download e não executar o processamento (*scraping*) ao mesmo tempo é concentrar o processo lento em uma etapa isolada, configurando uma boa prática. De fato, o processamento pode necessitar de várias flags e, de acordo com necessidades futuras, pode ser alterado. Se estivesse atrelado ao download, o mesmo teria que ou ser replicado fora do processo ou executado junto com um novo download, aplicando novamente um processo lento para uma mudança que deveria de ser rápida, uma vez que é uma alteração de processamento.

In [20]:
title_dataset = pd.read_csv(f"{BASE_PATH}/data/hqs/titles.csv")
driver = webdriver.Chrome()

# Verifica se existe um log de andamento de download
log_path = f"{BASE_PATH}/data/hqs/in_download.txt"
if not os.path.exists(log_path):
    with open(log_path, "w+") as file:
        file.write("")

already_downloaded = []
with open(log_path) as file:
    already_downloaded = [f.replace("\n", "").split("\t")[0] for f in file.readlines()]

for i in range(0, len(title_dataset)):
    # Recupera dados da publicação
    url = title_dataset["page_link"].values[i]
    name = title_dataset["name"].values[i].replace(":", " ").replace("?", "").replace("/", "-")
    start_time = time.time()

    # Verifica se ele já foi processado
    if name in already_downloaded:
        continue

    # Abre o driver e faz o download do código fonte
    driver.get(url)
    time.sleep(1)
    with open(f"{BASE_PATH}/data/hqs/download/{i} - {name}.html", "w+") as file:
        file.write(driver.page_source)

    # Registra no log o download
    total_time = time.time() - start_time
    with open(log_path, "a") as file:
        file.write(f"{name}\t{total_time}\n")

driver.close()

# Parte 3: Processamento

Tudo o que foi executado anteriormente teve por objetivo chegar nesta etapa. Visto que estamos lidando com páginas web que foram baixadas localmente em um diretório específico, este bloco vai iterar sobre todos os arquivos existentes no diretório e executar a raspagem, padronização e compilação dos dados. Neste ponto as particularidades de cada página interferem nos comandos, sendo necessário tratar todos os casos (página não encontrada, produto indisponível e apenas formato digital) para que se tenha o scrip mais genérico e eficiente possível.

Apesar de não ser tão lento quanto a Parte 2, este processo também requer um certo tempo de execução, que obviamente cresce com o aumento da quantidade de páginas baixadas, porém não em uma escala tão pronunciada. Ao final, o arquivo *prices_dataset.csv* é gerado com todos os dados perfeitamente processados e prontos para seguirem para os processos de análise estatística.

In [138]:
pages = os.listdir(f"{BASE_PATH}/data/hqs/download")
dataset = []

month_to_number = {
    "janeiro": "01", "fevereiro": "02", "março": "03", "abril": "04",
    "maio": "05", "junho": "06", "julho": "07", "agosto": "08",
    "setembro": "09", "outubro": "10", "novembro": "11", "dezembro": "12"
}

for page in pages:
    # Inicia um registro e acessa a página baixada
    register = dict()
    page_path = f"{BASE_PATH}/data/hqs/download/{page}"

    with open(page_path) as file:
        download_at = datetime.fromtimestamp(os.path.getctime(page_path)).isoformat()
        soup = BeautifulSoup(file, "html.parser")

    # Identifica se o livro está fora de estoque. Neste caso, a página não contêm
    # a informação de preço, que é fundamental para esta análise. Logo, este item
    # não vai compor o dataset final.
    if soup.find("div", {"id": "outOfStock"}) is not None:
        continue

    # Algumas páginas vieram sem conteúdo, desta forma não existe motivo para
    # processá-las. Esta flag remove estas páginas da lista de execução.
    if soup.find("title").text.strip() == "Não foi possível encontrar esta página":
        continue

    # Verifica se é um livro digital, encontrando qual versão está selecionada na
    # página. A seleção é uma div (cartão) com nome do formato e o menor preço.
    for element in soup.find_all("div", {"class": "a-ws-row"}):
        selected_format = element.find("div", {"class":"selected"})
        if selected_format is not None:
            break

    book_format = selected_format.find("span", {"class": "slot-title"})\
        .find("span").text.upper().strip()

    if book_format == "KINDLE":
        continue

    # Registra o formato encontrado. Importante que apenas formatos físicos são
    # incluídos neste conjunto de dados.
    register["format"] = book_format

    # Recupera data de acesso, que é a data em que foi feito o download da página.
    register["download_at"] = download_at

    # Recupera o preço de venda acessando o valor original (preço de capa). Em casos
    # em não existe preço de capa, apenas uma listagem de marketplace, utiliza o
    # preço exibido no cartão selecionado.
    element = soup.find("span", {"id": "listPrice"})
    if element is None:
        element = selected_format.find("span", {"class": "slot-price"})

    full_price = float(re.findall("[0-9]+,[0-9]+", element.text.strip())[0].replace(",", "."))
    register["full_price"] = full_price

    # Recupera o nome
    name = soup.find("span", {"id": "productTitle"}).text.strip()
    register["name"] = name

    # Organiza a lista de informações genéricas
    list_data = dict()
    elements = soup.find("div", {"id": "detailBullets_feature_div"})\
        .find_all("span", {"class": "a-list-item"})
    for element in elements:
        fields = element.text.replace("\n", " ").replace("\u200f", "")\
            .replace("\u200e", "").replace("  ", "").strip().split(":")
        if len(fields) < 2:
            continue
        list_data[fields[0].strip().upper()] = fields[1].strip()

    # Recupera o nome editora.
    publisher = list_data["EDITORA"].split(";")[0].split("(")[0]
    register["publisher"] = publisher.upper()

    # Recupera a data de lançamento da edição.
    i = list_data["EDITORA"].index("(")
    date_parts = list_data["EDITORA"][i+1:-1].split(" ")
    release_date = f"{date_parts[2]}-{month_to_number[date_parts[1]]}-{date_parts[0]}"
    register["release_date"] = release_date

    # Recupera o número de páginas da edição. Acessa uma div específica (cartão)
    # que exibe esta informação. A expressão regular captura somente o número. Se
    # não existir informação do número de páginas, ignora o registro, pois se trata
    # de um anúncio falho.
    element = soup.find("div", {"id": "rpi-attribute-book_details-fiona_pages"})
    if element is None:
        continue

    element.find("div", {"class": "a-section a-spacing-none a-text-center rpi-attribute-value"})\
        .find("span")
    register["pages"] = int(re.findall("[0-9]+", element.text)[0])

    # Recupera dimensões no formato altura x largura x profundidade. Caso não
    # exista a informação, atribui null
    dimension = None
    if "DIMENSÕES" in list_data.keys():
        dimension = list_data["DIMENSÕES"]
    register["dimensions"] = dimension

    # Recupera os registros de ISBN. Caso não existam, atribui null.
    isbn_10, isbn_13 = None, None
    if "ISBN-10" in list_data.keys():
        isbn_10 = list_data["ISBN-10"]

    if "ISBN-13" in list_data.keys():
        isbn_13 = list_data["ISBN-13"]

    register["ISBN-10"] = isbn_10
    register["ISBN-13"] = isbn_13

    # Recupera nota das avaliações dos clientes. Quando não existe avaliação, preenche
    # com o valor null.
    customer_review = None
    element = soup.find("div", {"id": "averageCustomerReviews"})
    if element is not None:
        element = element.find("span", {"class": "a-size-base a-color-base"})
        customer_review = float(element.text.strip().replace(",", "."))
    register["customers_review"] = customer_review

    # Recupera a descrição
    review = []
    for element in soup.find("div", {"id": "bookDescription_feature_div"}).children:
        review.append(element.text.replace("\n", " ").strip())
    register["about"] = " ".join(review).replace("  ", " ").strip()

    dataset.append(register)

# Organiza em dataframe e salva resultado
dataset = pd.DataFrame(dataset)
dataset.to_csv(f"{BASE_PATH}/data/hqs/prices_dataset.csv", index=False, sep="\t")